# Data import

In [1]:
import pandas as pd
df = pd.read_excel(r"C:\gamezone-orders-data.xlsx", sheet_name = None)
df.keys()

dict_keys(['orders', 'region'])

Let's create the dataframes and make a backup of the raw data.

In [2]:
df_orders = df['orders']
df_orders_raw = df_orders.copy()
df_regions = df['region']
df_regions_raw = df_regions.copy()

# Data cleaning

## Orders table

In [3]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21864 entries, 0 to 21863
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   USER_ID                  21864 non-null  object        
 1   ORDER_ID                 21864 non-null  object        
 2   PURCHASE_TS              21864 non-null  object        
 3   SHIP_TS                  21864 non-null  datetime64[ns]
 4   PRODUCT_NAME             21864 non-null  object        
 5   PRODUCT_ID               21864 non-null  object        
 6   USD_PRICE                21859 non-null  float64       
 7   PURCHASE_PLATFORM        21864 non-null  object        
 8   MARKETING_CHANNEL        21781 non-null  object        
 9   ACCOUNT_CREATION_METHOD  21781 non-null  object        
 10  COUNTRY_CODE             21826 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 1.8+ MB


There are several columns with the 'object' dtype and some null entries.

### Duplicates

Let's see if some lines are duplicated.

In [4]:
df_orders[df_orders.duplicated(keep=False)]

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
9378,b66cdb8d,7a5f67e18fa77291,2020-01-27 00:00:00,2020-01-28,27in 4K gaming monitor,e7e6,480.00,website,direct,desktop,US
9379,b66cdb8d,7a5f67e18fa77291,2020-01-27 00:00:00,2020-01-28,27in 4K gaming monitor,e7e6,480.00,website,direct,desktop,US
9563,6270d6f9,7d09de332e342684,2020-01-27 00:00:00,2020-01-30,27in 4K gaming monitor,891b,332.20,website,direct,desktop,BR
9564,6270d6f9,7d09de332e342684,2020-01-27 00:00:00,2020-01-30,27in 4K gaming monitor,891b,332.20,website,direct,desktop,BR
10051,3838f9e6,833086c869925765,2020-01-24 00:00:00,2020-01-25,27in 4K gaming monitor,891b,317.14,website,direct,desktop,JP
...,...,...,...,...,...,...,...,...,...,...,...
21105,c1c40ee1,f8b8868060282010,2020-01-23 00:00:00,2020-01-26,27in 4K gaming monitor,891b,480.00,website,direct,desktop,IN
21352,8b8daee8,fb26e69e0c859280,2020-01-30 00:00:00,2020-02-01,27in 4K gaming monitor,891b,312.00,website,direct,desktop,US
21353,8b8daee8,fb26e69e0c859280,2020-01-30 00:00:00,2020-02-01,27in 4K gaming monitor,891b,312.00,website,direct,desktop,US
21453,af7d9829,fc629d3dc5474356,2020-01-30 00:00:00,2020-02-02,27in 4K gaming monitor,891b,360.42,website,affiliate,unknown,US


There are a few, let's delete those.

In [5]:
df_orders.drop_duplicates(inplace = True)

### 1st column USER_ID

Let's check if there are non-string types in the values.

In [6]:
i = 0
while i < len(df_orders):
    if not isinstance(df_orders['USER_ID'].iloc[i], str):
        print(f"Index {i}, Format {type(df_orders['USER_ID'].iloc[i])}, Value: {df_orders['USER_ID'].iloc[i]}")
    i = i+1

Index 14, Format <class 'int'>, Value: 68673786
Index 74, Format <class 'int'>, Value: 14934774
Index 85, Format <class 'int'>, Value: 2858600000000000012200257257472
Index 103, Format <class 'int'>, Value: 83453728
Index 118, Format <class 'int'>, Value: 15827421
Index 134, Format <class 'int'>, Value: 23949611
Index 140, Format <class 'int'>, Value: 9321099999999999486643772227779884059866950819426060179294460368168198587821347206856704
Index 165, Format <class 'int'>, Value: 78548048
Index 166, Format <class 'int'>, Value: 78548048
Index 173, Format <class 'int'>, Value: 4087999999999999836946953680523207114752
Index 208, Format <class 'int'>, Value: 91703440
Index 209, Format <class 'int'>, Value: 91703440
Index 228, Format <class 'int'>, Value: 13462209
Index 255, Format <class 'int'>, Value: 50911000000000000000
Index 281, Format <class 'int'>, Value: 76211116
Index 282, Format <class 'int'>, Value: 24280780
Index 347, Format <class 'int'>, Value: 40134030
Index 392, Format <clas

We can see that all the non-string lines are in int format because they are composed exclusively of numbers.

In [7]:
df_orders['USER_ID'] = df_orders['USER_ID'].astype('string')

We also notice that some IDs are uncommonly long in comparison with the rest of the user IDs, let's make a list of all those lines to check with the (fictional) stakeholders at the source of this data.

In [8]:
long_userIDs_list = df_orders[df_orders['USER_ID'].apply(lambda x: len(x)>8)].copy()

### 2nd column ORDER_ID

Let's check if there are non-string types in the values.

In [9]:
i = 0
while i < len(df_orders):
    if not isinstance (df_orders['ORDER_ID'].iloc[i],str):
        print(f"Index {i}, Format {type(df_orders['ORDER_ID'].iloc[i])}, Value: {df_orders['ORDER_ID'].loc[i]}")
    i = i+1

All the values already are strings, we can thus convert the column.

In [10]:
df_orders['ORDER_ID'] = df_orders['ORDER_ID'].astype('string')

Logically, we shouldn't have any duplicate order IDs in this table. Let's check.

In [11]:
df_orders[df_orders['ORDER_ID'].duplicated(keep = False)]

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
9921,e80b93ad,815caec5eb998020,2020-01-30 00:00:00,2020-01-31,27in 4K gaming monitor,891b,408.20,website,email,mobile,DE
9922,e80b93ad,815caec5eb998020,2020-01-30 00:00:00,2020-01-31,27in 4K gaming monitor,891b,408.20,website,email,desktop,DE
13502,3bb56d71,add88586a5827183,2020-01-22 00:00:00,2020-01-23,27in 4K gaming monitor,891b,484.89,website,email,mobile,GB
13503,3bb56d71,add88586a5827183,2020-01-22 00:00:00,2020-01-23,27in 4K gaming monitor,891b,484.89,website,email,desktop,GB
18909,52e2062d,e44ebe4eed936564,2021-02-10 00:00:00,2021-02-11,Nintendo Switch,e682,102.88,website,direct,desktop,MX
...,...,...,...,...,...,...,...,...,...,...,...
21735,b0a2896,fecc9d6766231653,2021-02-22 00:00:00,2021-02-25,Nintendo Switch,8d0d,184.90,website,direct,desktop,GB
21742,e69c1f1e,fede5c24b0797422,2021-02-23 00:00:00,2021-02-25,Nintendo Switch,8d0d,142.80,website,direct,desktop,US
21743,9c12903,fede5c24b0797422,2021-02-23 00:00:00,2021-02-25,Nintendo Switch,8d0d,142.80,website,direct,desktop,US
21751,ebf6f56d,fef061c2a1499684,2021-02-06 00:00:00,2021-02-09,Nintendo Switch,e682,164.42,website,direct,desktop,ES


We unfortunately found duplicates with various differences in the other fields. As there is no way to identify which line is the right one, let's make a list of all those lines to check with the (fictional) stakeholders at the source of this data.

In [12]:
duplicate_orderID_list = df_orders[df_orders['ORDER_ID'].duplicated(keep = False)].copy()

Let's put it on the side and keep going.

### 3rd column PURCHASE_TS

Let's check if there are non-date types in the values and transform it to timestamp.

In [13]:
import datetime

In [14]:
i = 0
while i < len(df_orders):
    if not isinstance(df_orders['PURCHASE_TS'].iloc[i],datetime.datetime):
        print(f"Index {i}, Format {type(df_orders['PURCHASE_TS'].iloc[i])}, Value: {df_orders['PURCHASE_TS'].iloc[i]}")
    i = i+1

Index 1047, Format <class 'str'>, Value:   
Index 1251, Format <class 'str'>, Value: 12-31-2020 01:54:35
Index 5846, Format <class 'str'>, Value: 06-14-2019 13:62:38
Index 7363, Format <class 'str'>, Value: 12-31-2020 01:54:34
Index 9715, Format <class 'str'>, Value: 12-31-2020 01:54:33
Index 10009, Format <class 'str'>, Value: 12-31-2020 01:54:44
Index 10886, Format <class 'str'>, Value: 12-31-2020 01:54:40
Index 11847, Format <class 'str'>, Value: 06-14-2019 13:62:39
Index 14561, Format <class 'str'>, Value: 12-31-2020 01:54:39
Index 16141, Format <class 'str'>, Value: 06-14-2019 13:62:34
Index 20693, Format <class 'str'>, Value: 06-14-2019 13:62:41


In [15]:
i = 0
while i < len(df_orders):
    if not isinstance(df_orders['PURCHASE_TS'].iloc[i],datetime.datetime):
        print(f"Index {i}, Current value: {df_orders['PURCHASE_TS'].iloc[i]}, New timestamp value : {pd.to_datetime(df_orders['PURCHASE_TS'].iloc[i].split(' ')[0], errors = 'coerce')}")
    i = i+1

Index 1047, Current value:   , New timestamp value : NaT
Index 1251, Current value: 12-31-2020 01:54:35, New timestamp value : 2020-12-31 00:00:00
Index 5846, Current value: 06-14-2019 13:62:38, New timestamp value : 2019-06-14 00:00:00
Index 7363, Current value: 12-31-2020 01:54:34, New timestamp value : 2020-12-31 00:00:00
Index 9715, Current value: 12-31-2020 01:54:33, New timestamp value : 2020-12-31 00:00:00
Index 10009, Current value: 12-31-2020 01:54:44, New timestamp value : 2020-12-31 00:00:00
Index 10886, Current value: 12-31-2020 01:54:40, New timestamp value : 2020-12-31 00:00:00
Index 11847, Current value: 06-14-2019 13:62:39, New timestamp value : 2019-06-14 00:00:00
Index 14561, Current value: 12-31-2020 01:54:39, New timestamp value : 2020-12-31 00:00:00
Index 16141, Current value: 06-14-2019 13:62:34, New timestamp value : 2019-06-14 00:00:00
Index 20693, Current value: 06-14-2019 13:62:41, New timestamp value : 2019-06-14 00:00:00


In [16]:
i = 0
while i < len(df_orders):
    if not isinstance(df_orders['PURCHASE_TS'].iloc[i],datetime.datetime):
        df_orders.loc[df_orders.index[i],'PURCHASE_TS'] = pd.to_datetime(df_orders['PURCHASE_TS'].iloc[i].split(' ')[0], errors = 'coerce')
    else:
        df_orders.loc[df_orders.index[i],'PURCHASE_TS'] = pd.to_datetime(df_orders['PURCHASE_TS'].iloc[i])
    i = i+1

In [17]:
df_orders['PURCHASE_TS'] = pd.to_datetime(df_orders['PURCHASE_TS'], errors='coerce')

In [18]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21829 entries, 0 to 21863
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   USER_ID                  21829 non-null  string        
 1   ORDER_ID                 21829 non-null  string        
 2   PURCHASE_TS              21828 non-null  datetime64[ns]
 3   SHIP_TS                  21829 non-null  datetime64[ns]
 4   PRODUCT_NAME             21829 non-null  object        
 5   PRODUCT_ID               21829 non-null  object        
 6   USD_PRICE                21824 non-null  float64       
 7   PURCHASE_PLATFORM        21829 non-null  object        
 8   MARKETING_CHANNEL        21746 non-null  object        
 9   ACCOUNT_CREATION_METHOD  21746 non-null  object        
 10  COUNTRY_CODE             21791 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(6), string(2)
memory usage: 2.5+ MB


There is one null value in this column. Let's ignore it as the impact is very limited.

### 4th column SHIP_TS

This column doesn't have nulls and is already clean.

In [19]:
df_orders[df_orders['SHIP_TS'] < df_orders['PURCHASE_TS']]

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
0,2c06175e,0001328c3c220830,2020-12-24,2020-12-13,Nintendo Switch,e682,168.00,website,affiliate,unknown,US
1,ee8e5bc2,0002af7a5c6100772,2020-10-01,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,DE
2,9eb4efe0,0002b8350e167074,2020-04-21,2020-02-16,Nintendo Switch,8d0d,151.20,website,direct,desktop,US
3,cac7cbaf,0006d06b98385729,2020-04-07,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,AU
4,6b0230bc,00097279a2f46150,2020-11-24,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,TR
...,...,...,...,...,...,...,...,...,...,...,...
10890,6dcd78e9,8d6795b3b1447377,2020-12-31,2020-02-20,27in 4K gaming monitor,891b,408.00,website,direct,mobile,US
11853,2fa9f33d,99d824517da22388,2019-06-14,2019-04-11,JBL Quantum 100 Gaming Headset,ab0f,21.19,website,direct,mobile,JP
14576,43bbad06,ba7f92bd22730500,2020-12-31,2019-03-31,JBL Quantum 100 Gaming Headset,ab0f,24.00,website,direct,desktop,US
16163,b313cea5,c9e0aea0d9a75871,2019-06-14,2019-05-18,JBL Quantum 100 Gaming Headset,ab0f,19.20,website,direct,desktop,US


It seems however that there are many lines in which the shipping date is before the purchasing date, which doesn't make sense. Let's make a list of all those lines to check with the (fictional) stakeholders at the source of this data.

In [20]:
shipping_before_purchase_list = df_orders[df_orders['SHIP_TS'] < df_orders['PURCHASE_TS']].copy()

### 5th column PRODUCT_NAME

Let's check if there are non-string types in the values.

In [21]:
i=0
while i < len(df_orders):
    if not isinstance(df_orders['PRODUCT_NAME'].iloc[i],str):
        print(f"Index {i}, Format {type(df_orders['PRODUCT_NAME'].iloc[i])}, Value {df_orders['PRODUCT_NAME'].iloc[i]}")
    i = i+1

All the values are string types, we can thus convert the column to string dtype.

In [22]:
df_orders['PRODUCT_NAME'] = df_orders['PRODUCT_NAME'].astype('string')

Let's have a look at the product name list.

In [23]:
df_orders['PRODUCT_NAME'].unique()

<StringArray>
[               'Nintendo Switch',      'Sony PlayStation 5 Bundle',
         '27in 4K gaming monitor', 'JBL Quantum 100 Gaming Headset',
              'Dell Gaming Mouse',     'Acer Nitro V Gaming Laptop',
        'Lenovo IdeaPad Gaming 3',       'Razer Pro Gaming Headset',
     '27inches 4k gaming monitor']
Length: 9, dtype: string

The products '27in 4K gaming monitor' and '27inches 4k gaming monitor' seem to be the same, so let's standardize the names by choosing one and renaming the other accordingly.

In [24]:
df_orders.loc[df_orders['PRODUCT_NAME'] == '27in 4K gaming monitor', 'PRODUCT_NAME'] = '27inches 4k gaming monitor'

### 6th column PRODUCT_ID

Let's check if there are non-string types in the values.

In [25]:
i=0
while i < len(df_orders):
    if not isinstance(df_orders['PRODUCT_ID'].iloc[i],str):
        print(f"Index {i}, Format {type(df_orders['PRODUCT_ID'].iloc[i])}, Value {df_orders['PRODUCT_ID'].iloc[i]}")
    i = i+1

Index 21, Format <class 'int'>, Value 2997
Index 59, Format <class 'int'>, Value 8315
Index 89, Format <class 'int'>, Value 5142
Index 90, Format <class 'int'>, Value 5142
Index 91, Format <class 'int'>, Value 5142
Index 125, Format <class 'int'>, Value 8315
Index 134, Format <class 'int'>, Value 8315
Index 216, Format <class 'int'>, Value 1238
Index 224, Format <class 'int'>, Value 2997
Index 225, Format <class 'int'>, Value 2997
Index 243, Format <class 'int'>, Value 2997
Index 363, Format <class 'int'>, Value 2997
Index 392, Format <class 'int'>, Value 8315
Index 440, Format <class 'int'>, Value 8315
Index 448, Format <class 'int'>, Value 8315
Index 472, Format <class 'int'>, Value 5142
Index 497, Format <class 'int'>, Value 2997
Index 511, Format <class 'int'>, Value 2997
Index 526, Format <class 'int'>, Value 8315
Index 549, Format <class 'int'>, Value 8315
Index 550, Format <class 'int'>, Value 8315
Index 578, Format <class 'int'>, Value 7416
Index 599, Format <class 'int'>, Valu

We can see that all the non-string lines are in int format because they are composed exclusively of numbers.

In [26]:
df_orders['PRODUCT_ID'] = df_orders['PRODUCT_ID'].astype('string')

Let's check the coherence with the PRODUCT_NAME column.

In [27]:
df_orders[['PRODUCT_NAME','PRODUCT_ID']].drop_duplicates().sort_values(by = 'PRODUCT_ID')

,PRODUCT_NAME,PRODUCT_ID
3228,Nintendo Switch,03ca
182,Lenovo IdeaPad Gaming 3,04ac
127,Dell Gaming Mouse,0c5a
1256,Nintendo Switch,0d23
216,27inches 4k gaming monitor,1238
13736,Sony PlayStation 5 Bundle,12b1
107,Acer Nitro V Gaming Laptop,22ea
8088,Nintendo Switch,24c1
21,JBL Quantum 100 Gaming Headset,2997
615,27inches 4k gaming monitor,2a50


It seems that PRODUCT_NAME has a multiple to one relationship with PRODUCT_ID which can make sense.

### 7th column USD_PRICE

This column is already in float format, let's then check if there is any strange values.

We already saw in the df_orders.info() that there were null values in this column, let's make a list of all those lines to check with the (fictional) stakeholders at the source of this data.

In [28]:
zero_dollars_orders = df_orders[(df_orders['USD_PRICE']==0) | (df_orders['USD_PRICE'].isnull())].copy()
len(zero_dollars_orders)

34

Note that it represents only 34 rows, thus not having much impact on any analysis. Let's delete those lines and keep going.

In [29]:
df_orders = df_orders[df_orders['USD_PRICE']!=0].copy()
df_orders = df_orders[df_orders['USD_PRICE'].notnull()].copy()

Let's check the highest and lowest values.

In [30]:
df_orders.sort_values(by = 'USD_PRICE', ascending = True).head(10)

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
9693,f1e47071,7e8dab4db9e102226,2020-12-09,2020-12-12,JBL Quantum 100 Gaming Headset,ab0f,6.11,website,direct,desktop,TR
19068,fd5574ab,e5c64fcd689107006,2020-12-02,2020-12-03,JBL Quantum 100 Gaming Headset,ab0f,6.11,website,direct,desktop,TR
728,8e8cda8f,09bb555536760536,2020-09-26,2020-08-10,JBL Quantum 100 Gaming Headset,ab0f,6.11,website,email,desktop,TR
10315,d09fc296,869c79f9fe188110,2020-11-12,2020-11-14,JBL Quantum 100 Gaming Headset,ab0f,6.32,website,direct,desktop,TR
12464,493bddfd,a195c59620b32614,2020-11-08,2020-11-09,JBL Quantum 100 Gaming Headset,ab0f,6.55,website,email,desktop,TR
14679,ef3bcb29,bb69368fa16101075,2021-01-25,2021-01-28,JBL Quantum 100 Gaming Headset,ab0f,6.63,website,direct,desktop,TR
16184,49ae0a41,ca16e85211432807,2020-09-24,2020-09-26,JBL Quantum 100 Gaming Headset,ab0f,6.77,website,direct,desktop,TR
10387,e56b4114,8770efef34996921,2019-05-14,2019-05-17,JBL Quantum 100 Gaming Headset,2997,7.14,mobile app,email,mobile,TR
2004,b86a425c,1a32257ae2f78125,2020-06-06,2020-06-07,JBL Quantum 100 Gaming Headset,ab0f,7.19,website,direct,desktop,TR
11578,69612212,965eeb1c87b1826,2020-07-09,2020-07-11,JBL Quantum 100 Gaming Headset,ab0f,7.23,website,direct,mobile,TR


In [31]:
df_orders.sort_values(by = 'USD_PRICE', ascending = False).head(10)

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
19196,47c3a16b,e7023c6eb9032084,2019-05-19,2019-05-20,Sony PlayStation 5 Bundle,e22d,3146.88,website,direct,desktop,GB
12353,5cc9065d,a0306bced5a40489,2019-01-28,2019-01-30,Sony PlayStation 5 Bundle,54ed,2509.67,website,direct,mobile,GB
11748,741a663e,9867234962049857,2019-02-20,2019-02-23,Sony PlayStation 5 Bundle,54ed,2308.32,website,direct,mobile,IT
13829,9ffc43cc,b20616b63f167635,2020-12-03,2020-12-06,Sony PlayStation 5 Bundle,54ed,2250.93,website,direct,desktop,GB
6756,5dbb56ac,5920e517ea640884,2020-11-19,2020-11-20,Sony PlayStation 5 Bundle,54ed,2250.93,website,direct,desktop,GB
13990,67492bfb,b3eace9b2f344608,2019-04-27,2019-04-30,Sony PlayStation 5 Bundle,54ed,2236.25,website,direct,desktop,NL
8710,60e89e69,71fdc2fab6142145,2019-03-06,2019-03-07,Sony PlayStation 5 Bundle,54ed,2205.58,website,direct,desktop,DK
20154,3a8bed4a,efe3ad7eacf26675,2019-07-27,2019-07-29,Sony PlayStation 5 Bundle,54ed,2195.59,website,direct,desktop,DE
20155,3a8bed4a,efe3ad7eacf26676,2019-07-27,2019-07-29,Sony PlayStation 5 Bundle,54ed,2195.59,website,direct,desktop,DE
20311,ed6518c2,f138c5e035d100288,2019-10-04,2019-10-07,Sony PlayStation 5 Bundle,54ed,2182.90,website,direct,desktop,FR


The highest and lowest values seem realistic.

### 8th column PURCHASE_PLATFORM

Let's check if there are non-string types in the values.

In [32]:
i=0
while i < len(df_orders):
    if not isinstance(df_orders['PURCHASE_PLATFORM'].iloc[i],str):
        print(f"Index {i}, Format {type(df_orders['PURCHASE_PLATFORM'].iloc[i])}, Value {df_orders['PURCHASE_PLATFORM'].iloc[i]}")
    i = i+1

All the values are string types, we can thus convert the column to string dtype.

In [33]:
df_orders['PURCHASE_PLATFORM'] = df_orders['PURCHASE_PLATFORM'].astype('string')

Let's have a look at the purchase platform name list.

In [34]:
df_orders['PURCHASE_PLATFORM'].unique()

<StringArray>
['website', 'mobile app']
Length: 2, dtype: string

Everything looks good for this column.

### 9th column MARKETING_CHANNEL

Let's check if there are non-string types in the values.

In [35]:
i=0
while i < len(df_orders):
    if not isinstance(df_orders['MARKETING_CHANNEL'].iloc[i],str):
        print(f"Index {i}, Format {type(df_orders['MARKETING_CHANNEL'].iloc[i])}, Value {df_orders['MARKETING_CHANNEL'].iloc[i]}")
    i = i+1

Index 243, Format <class 'float'>, Value nan
Index 563, Format <class 'float'>, Value nan
Index 783, Format <class 'float'>, Value nan
Index 1494, Format <class 'float'>, Value nan
Index 1641, Format <class 'float'>, Value nan
Index 1861, Format <class 'float'>, Value nan
Index 2481, Format <class 'float'>, Value nan
Index 2985, Format <class 'float'>, Value nan
Index 3457, Format <class 'float'>, Value nan
Index 3591, Format <class 'float'>, Value nan
Index 3592, Format <class 'float'>, Value nan
Index 3651, Format <class 'float'>, Value nan
Index 3958, Format <class 'float'>, Value nan
Index 4009, Format <class 'float'>, Value nan
Index 4247, Format <class 'float'>, Value nan
Index 4413, Format <class 'float'>, Value nan
Index 4491, Format <class 'float'>, Value nan
Index 4905, Format <class 'float'>, Value nan
Index 5309, Format <class 'float'>, Value nan
Index 5515, Format <class 'float'>, Value nan
Index 5638, Format <class 'float'>, Value nan
Index 5672, Format <class 'float'>, V

The null values of this column are considered as float.

Let's convert the column to string dtype.

In [36]:
df_orders['MARKETING_CHANNEL'] = df_orders['MARKETING_CHANNEL'].astype('string')

Let's look at the list of marketing channels.

In [37]:
df_orders['MARKETING_CHANNEL'].unique()

<StringArray>
['affiliate', 'direct', 'email', 'social media', <NA>, 'unknown']
Length: 6, dtype: string

We can see that there is a category 'unknown', let's attribute all the null values to that category.

In [38]:
df_orders.loc[df_orders['MARKETING_CHANNEL'].isnull(), 'MARKETING_CHANNEL'] = 'unknown'

### 10th column ACCOUNT_CREATION_METHOD

Let's check if there are non-string types in the values.

In [39]:
i=0
while i < len(df_orders):
    if not isinstance(df_orders['ACCOUNT_CREATION_METHOD'].iloc[i],str):
        print(f"Index {i}, Format {type(df_orders['ACCOUNT_CREATION_METHOD'].iloc[i])}, Value {df_orders['ACCOUNT_CREATION_METHOD'].iloc[i]}")
    i = i+1

Index 243, Format <class 'float'>, Value nan
Index 563, Format <class 'float'>, Value nan
Index 783, Format <class 'float'>, Value nan
Index 1494, Format <class 'float'>, Value nan
Index 1641, Format <class 'float'>, Value nan
Index 1861, Format <class 'float'>, Value nan
Index 2481, Format <class 'float'>, Value nan
Index 2985, Format <class 'float'>, Value nan
Index 3457, Format <class 'float'>, Value nan
Index 3591, Format <class 'float'>, Value nan
Index 3592, Format <class 'float'>, Value nan
Index 3651, Format <class 'float'>, Value nan
Index 3958, Format <class 'float'>, Value nan
Index 4009, Format <class 'float'>, Value nan
Index 4247, Format <class 'float'>, Value nan
Index 4413, Format <class 'float'>, Value nan
Index 4491, Format <class 'float'>, Value nan
Index 4905, Format <class 'float'>, Value nan
Index 5309, Format <class 'float'>, Value nan
Index 5515, Format <class 'float'>, Value nan
Index 5638, Format <class 'float'>, Value nan
Index 5672, Format <class 'float'>, V

The null values of this column are considered as float.

Let's convert the column to string dtype.

In [40]:
df_orders['ACCOUNT_CREATION_METHOD'] = df_orders['ACCOUNT_CREATION_METHOD'].astype('string')

Let's look at the list of account creation methods.

In [41]:
df_orders['ACCOUNT_CREATION_METHOD'].unique()

<StringArray>
['unknown', 'desktop', 'mobile', 'tablet', <NA>, 'tv']
Length: 6, dtype: string

We can see that there is a category 'unknown', let's attribute all the null values to that category.

In [42]:
df_orders.loc[df_orders['ACCOUNT_CREATION_METHOD'].isnull(), 'ACCOUNT_CREATION_METHOD'] = 'unknown'

### 11th column COUNTRY_CODE

Let's check if there are non-string types in the values.

In [43]:
i=0
while i < len(df_orders):
    if not isinstance(df_orders['COUNTRY_CODE'].iloc[i],str):
        print(f"Index {i}, Format {type(df_orders['COUNTRY_CODE'].iloc[i])}, Value {df_orders['COUNTRY_CODE'].iloc[i]}")
    i = i+1

Index 526, Format <class 'float'>, Value nan
Index 671, Format <class 'float'>, Value nan
Index 1043, Format <class 'float'>, Value nan
Index 3581, Format <class 'float'>, Value nan
Index 4079, Format <class 'float'>, Value nan
Index 4080, Format <class 'float'>, Value nan
Index 4870, Format <class 'float'>, Value nan
Index 5139, Format <class 'float'>, Value nan
Index 6114, Format <class 'float'>, Value nan
Index 6115, Format <class 'float'>, Value nan
Index 6507, Format <class 'float'>, Value nan
Index 6508, Format <class 'float'>, Value nan
Index 6974, Format <class 'float'>, Value nan
Index 6989, Format <class 'float'>, Value nan
Index 7185, Format <class 'float'>, Value nan
Index 7787, Format <class 'float'>, Value nan
Index 9238, Format <class 'float'>, Value nan
Index 9278, Format <class 'float'>, Value nan
Index 9832, Format <class 'float'>, Value nan
Index 9890, Format <class 'float'>, Value nan
Index 10719, Format <class 'float'>, Value nan
Index 14609, Format <class 'float'>

The null values of this column are considered as float.

Let's convert the column to string dtype.

In [44]:
df_orders['COUNTRY_CODE'] = df_orders['COUNTRY_CODE'].astype('string')

As there is no way to infer the missing country codes with the information we have, let's make a list of all those lines to check with the (fictional) stakeholders at the source of this data.

In [45]:
missing_country_code = df_orders[df_orders['COUNTRY_CODE'].isnull()].copy()
len(missing_country_code)

38

Note that it represents only 38 rows, thus not affecting too much our analysis. Let's put it on the side and keep going.

## Regions table

In [46]:
df_regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   COUNTRY_CODE  191 non-null    object
 1   REGION        169 non-null    object
dtypes: object(2)
memory usage: 3.1+ KB


Both column have the type object and have null values.

### Duplicates

Let's see if some lines are duplicated.

In [47]:
df_regions[df_regions.duplicated(keep=False)]

,COUNTRY_CODE,REGION


There is no duplicated lines.

### 1st column COUNTRY_CODE

Let's check the null value we identified above.

In [48]:
df_regions[df_regions['COUNTRY_CODE'].isnull()]

,COUNTRY_CODE,REGION
129,NaN,EMEA


This table seems to be is ordered alphabetically by the COUNTRY_CODE column. From the index of the line with the null COUNTRY_CODE, we could infer that the data that should be there is alphabetically between the country code of line 128 and 130.

In [49]:
df_regions.loc[128:130]

,COUNTRY_CODE,REGION
128,MZ,EMEA
129,NaN,EMEA
130,NC,APAC


After a quick search over the internet, it seems that the only country with a code that fits is NA (Namibia). Let's thus replace the null value accordingly.

In [50]:
df_regions.loc[129,'COUNTRY_CODE'] = 'NA'

Let's convert the column to string dtype.

In [51]:
df_regions['COUNTRY_CODE'] = df_regions['COUNTRY_CODE'].astype('string')

### 2nd column REGION

Let's check the list of regions to understand the classification logic.

In [52]:
df_regions['REGION'].unique()

array(['EMEA', nan, 'APAC', 'LATAM', 'North America', 'X.x'], dtype=object)

Let's check the null and 'X.x' values.

In [53]:
df_regions[(df_regions['REGION'].isnull()) | (df_regions['REGION'] == 'X.x')]

,COUNTRY_CODE,REGION
2,AG,NaN
3,AI,NaN
25,BQ,NaN
27,BS,NaN
44,CW,NaN
61,GD,NaN
67,GL,NaN
69,GP,NaN
80,IE,NaN
89,JM,NaN


We can easily associate those country code to the right region by checking on the internet. Let's do that.

In [54]:
df_regions.loc[df_regions['COUNTRY_CODE']=='AG','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='AI','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='BQ','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='BS','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='CW','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='GD','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='GL','REGION'] = 'EMEA'
df_regions.loc[df_regions['COUNTRY_CODE']=='GP','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='IE','REGION'] = 'EMEA'
df_regions.loc[df_regions['COUNTRY_CODE']=='JM','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='KN','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='KY','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='LB','REGION'] = 'EMEA'
df_regions.loc[df_regions['COUNTRY_CODE']=='LC','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='MH','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='MQ','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='PG','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='PR','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='SX','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='TC','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='TT','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='US','REGION'] = 'North America'
df_regions.loc[df_regions['COUNTRY_CODE']=='VC','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='VG','REGION'] = 'LATAM'
df_regions.loc[df_regions['COUNTRY_CODE']=='VI','REGION'] = 'LATAM'

In [55]:
df_regions['REGION'] = df_regions['REGION'].astype('string')

The tables are now cleaned and usable for analysis.